In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os
from linkedin_jobs_scraper.utils.chrome_driver import build_driver
import os
import logging
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re
cookie = os.environ['LI_AT_COOKIE']
# company_job_link = "https://www.linkedin.com/company/triarq-health/jobs/"

In [60]:
# Transfer different country web name to www. 
def transform_url(url):
    return re.sub(r"https://\w+\.linkedin\.com", "https://www.linkedin.com", url)


def get_linkedin_company_url(company_name):
    search_query = f"{company_name} LinkedIn"
    google_search_url = f"https://www.google.com/search?q={search_query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(google_search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for link in soup.find_all('a'):
        url = link.get('href')
        if url is None:
            continue
        if 'linkedin.com/company' in url:
            url =  transform_url(url)
            linkedin_company_name = url.split('/')[-1]
            company_job_link = f"{url}/jobs/"
            company_data = {'company_name': company_name,
                'company_linkedin_name':linkedin_company_name, 
                'company_page_url':company_job_link}
            print(company_job_link)

            return company_data
    
    return "LinkedIn company page not found"

def get_jobs_page_link(company_data, driver, headless = True):
    company_job_link =  company_data['company_page_url']
    driver.get(company_job_link)
    print(company_job_link)
    # Wait for the page to load (optional)
    time.sleep(9)
    try:
        job_card = driver.find_element(By.XPATH, "//div[contains(@class, 'org-jobs-recently-posted-jobs-module__show-all-jobs-btn')]")
        link_botton = job_card.find_element(By.XPATH, ".//a")
        link_to_job_page = link_botton.get_attribute("href")
        print(f"msg:found job posted {link_to_job_page}")
        # driver.close()
        return link_to_job_page
    except Exception as e1:
        try:
            print(f"jobs container not found")
            no_jobs_element = driver.find_element(By.XPATH, "//div[contains(@class, 'org-jobs-empty-jobs-module artdeco-card')]")
            print(f"msg:no job posted")
            # driver.close()
            return None
        except Exception as e2:
            if 'unavailable' in driver.current_url.split('/'):
                print('msg: LinkedIn Page Expire and unavailable')
            else:
                print(f"msg:unknown error from {company_job_link}")
            # driver.close()
            return None
        
def company_job_link(company_name, driver):
    
    company_data = get_linkedin_company_url(company_name)

    if isinstance(company_data, str):
        print(f'{company_name} linkedIn Page Not found ')
        return {'company_name': company_name,'company_page': None, 'job_link': None}
    else:
        job_page_link = get_jobs_page_link(company_data, driver = driver)
        print(f'{company_name} done')
        return {'company_name': company_name, 'company_page': company_data['company_page_url'], 'job_link': job_page_link}
    

In [64]:
driver = build_driver(headless = False)
driver.get('https://www.linkedin.com')
driver.add_cookie({
                        'name': 'li_at',
                        'value': os.environ['LI_AT_COOKIE'],
                        'domain': '.www.linkedin.com'
                    })
company_data = get_linkedin_company_url("Biolojic Design")
job_page_link = get_jobs_page_link(company_data, driver = driver)
#driver.quit()

https://www.linkedin.com/company/biolojic-design-ltd/jobs/
https://www.linkedin.com/company/biolojic-design-ltd/jobs/
msg:found job posted https://www.linkedin.com/jobs/search/?f_C=18152425&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3939964795%2C3944970486%2C3944973145%2C3944971354


In [51]:
driver.quit()

In [13]:
rollup = pd.read_csv('Rollup.csv')
csv_file = 'results.csv'
# with open(csv_file, mode='w', newline='') as file:
#     writer = csv.DictWriter(file, fieldnames=['company_name', 'company_page', 'job_link'])
#     writer.writeheader()

# Function to append each result to the CSV file
def append_to_csv(result, csv_file):
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['company_name', 'company_page', 'job_link'])
        writer.writerow(result)


In [16]:
driver = build_driver(headless = True)
driver.get('https://www.linkedin.com')
driver.add_cookie({
                        'name': 'li_at',
                        'value': os.environ['LI_AT_COOKIE'],
                        'domain': '.www.linkedin.com'
                    })

for company in rollup['Company Name']:
    print(company)
    result = company_job_link(company, driver)
    print(result)
    append_to_csv(result, csv_file)
    
driver.quit()

Perception Neuroscience, Inc.
jobs container not found
msg:no job posted
Perception Neuroscience, Inc. done
{'company_name': 'Perception Neuroscience, Inc.', 'company_page': 'https://www.linkedin.com/company/perception-neurosciences/jobs/', 'job_link': None}
Solera Health
msg:found job posted https://www.linkedin.com/jobs/search/?f_C=6640471&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3971570718%2C3973575366
Solera Health done
{'company_name': 'Solera Health', 'company_page': 'https://www.linkedin.com/company/solera-health/jobs/', 'job_link': 'https://www.linkedin.com/jobs/search/?f_C=6640471&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3971570718%2C3973575366'}
Benuvia Manufacturing
jobs container not found
msg:no job posted
Benuvia Manufacturing done
{'company_name': 'Benuvia Manufacturing', 'company_page': 'https://www.linkedin.com/company/benuvia-operations-llc/jobs/', 'job_link': None}
Nova Discovery


KeyboardInterrupt: 